# Wrangling XML

In [1]:
with open('CustomersOrdersInNamespace.xml') as f:
    xmlcontent = f.read()

In [2]:
#!pip install xmltodict

In [3]:
import xmltodict

In [4]:
import json
import pandas as pd

In [5]:
mydict = xmltodict.parse(xmlcontent)

In [6]:
type(mydict)

collections.OrderedDict

In [7]:
custlist = mydict['Root']['Customers']['Customer']

In [8]:
buyer = mydict['Root']['Customers']['Customer'][0]
buyer

OrderedDict([('@CustomerID', 'GREAL'),
             ('CompanyName', 'Great Lakes Food Market'),
             ('ContactName', 'Howard Snyder'),
             ('ContactTitle', 'Marketing Manager'),
             ('Phone', '(503) 555-7555'),
             ('FullAddress',
              OrderedDict([('Address', '2732 Baker Blvd.'),
                           ('City', 'Eugene'),
                           ('Region', 'OR'),
                           ('PostalCode', '97403'),
                           ('Country', 'USA')]))])

In [9]:
buyer['FullAddress']

OrderedDict([('Address', '2732 Baker Blvd.'),
             ('City', 'Eugene'),
             ('Region', 'OR'),
             ('PostalCode', '97403'),
             ('Country', 'USA')])

In [ ]:
#Important idiom on how to go over key,value pairs in a dictionary!
for key,value in buyer['FullAddress'].items():
    print(f'key:\t{key},   \tvalue:\t{value}')

In [ ]:
for key,value in buyer['FullAddress'].items():
    buyer[key] = value
buyer

In [ ]:
# https://stackoverflow.com/questions/11277432/how-to-remove-a-key-from-a-python-dictionary
buyer.pop('FullAddress')
# Returns what we popped but we do not need it anymore

In [ ]:
buyer

In [ ]:
for cust in custlist[1:]: #The first one with index 0 we already processed!
    for key,value in cust['FullAddress'].items(): # could use a check if such a key exists first
        buyer[key] = value
    cust.pop('FullAddress')
custlist
    

In [ ]:
df = pd.DataFrame(custlist)
df

In [ ]:
orders =  mydict['Root']['Orders']['Order']
len(orders),orders[:3]

In [ ]:
def flatten_Dict_to_DF(dictlist,masterkey):
    '''
    Flattens dictionary within each dictionary in a list
    dictlist[index][masterkey] should contain the dictionary to be flattened
    original masterkey holding the child dictionary is destroyed
    Returns Pandas dataframe created from the list of dictionaries
    '''
    
    for rootdict in dictlist:
        if masterkey not in rootdict: #We might want to add a log or print to show that no key was found
            continue
        for key,value in rootdict[masterkey].items(): # could use a check if such a key exists first
            if key in rootdict:
                print(f'Careful, we do not want to overwrite this key{key}.')
            rootdict[key] = value
        rootdict.pop(masterkey)
        
    return pd.DataFrame(dictlist)
        
    

In [ ]:
flatten_Dict_to_DF(mydict['Root']['Orders']['Order'],'ShipInfo')

In [ ]:
oframe = flatten_Dict_to_DF(mydict['Root']['Orders']['Order'],'ShipInfo')
oframe

In [ ]:
bframe = flatten_Dict_to_DF(custlist, 'FullAddress')
bframe

In [ ]:
writer = pd.ExcelWriter('AdventureShop.xlsx') # could use with syntax

In [ ]:
bframe.to_excel(writer, 'Sheet1')

In [ ]:
oframe.to_excel(writer, 'Sheet2')

In [ ]:
writer.save()

In [ ]:
# This file can be sent to email automaticaly with library smtplib.

import smtplib